In [74]:
import importlib
import argparse
import os
import sys
import time
import wandb

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader

import random
import numpy as np
import yaml
from box import Box
from pprint import pprint
import wandb
from tqdm import tqdm
from datetime import timedelta
from collections import defaultdict

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import numpy as np

from einops import rearrange, repeat
from einops.layers.torch import Rearrange

CONFIG_FILENAME = '/home/liranc6/ecg_forecasting/liran_project/mrdiff/src/config_ecg.yml'

assert CONFIG_FILENAME.endswith('.yml')

with open(CONFIG_FILENAME, 'r') as file:
    config = yaml.safe_load(file)

# Add the parent directory to the sys.path
ProjectPath = config['project_path']
sys.path.append(ProjectPath)

import liran_project
from liran_project.mrdiff.src.parser import parse_args
from liran_project.utils.dataset_loader import SingleLeadECGDatasetCrops_mrDiff as DataSet
from liran_project.utils.util import ecg_signal_difference
from liran_project.mrdiff.exp_main import Exp_Main
from liran_project.utils.common import *

# Add the directory containing the exp module to the sys.path
exp_module_path = os.path.join(ProjectPath, 'mrDiff')
sys.path.append(exp_module_path)

# from mrDiff.exp.exp_main import Exp_Main
from mrDiff.data_process.etth_dataloader import Dataset_ETT_hour, Dataset_ETT_minute, Dataset_Custom, Dataset_Wind, Dataset_Caiso, Dataset_Production, Dataset_Caiso_M, Dataset_Production_M
from mrDiff.data_process.financial_dataloader import DatasetH
from mrDiff.data_process.forecast_dataloader import ForecastDataset
from mrDiff.exp.exp_basic import Exp_Basic
from mrDiff.models_diffusion import DDPM
from mrDiff.utils.tools import EarlyStopping, adjust_learning_rate, visual
from mrDiff.utils.metrics import metric

from liran_project.mrdiff.src.parser import Args

In [75]:
args = Args(CONFIG_FILENAME)

# Now you can use args as needed
vars(args)

{'configs_filename': '/home/liranc6/ecg_forecasting/liran_project/mrdiff/src/config_ecg.yml',
 'configs': Box({'project_path': '/home/liranc6/ecg_forecasting', 'tqdm': 'terminal', 'debug': False, 'resume_exp': {'resume': False, 'resume_from': 9, 'resume_optimizer': True, 'resume_epoch': 9, 'resume_metrics': True, 'resume_scheduler': True, 'resume_configuration': False, 'specific_chpt_path': '/home/liranc6/ecg_forecasting/liran_project/results/icentia11k/mrDiff/DDPM_icentia11k_ftS_sl105000_ll7500_pl7500_lr0.001_bs16_invFalse_itr0/21_10_2024_1424/logs/1_last_checkpoint.pth', 'was_resumed': False, 'model_start_training_time': 'None'}, 'wandb': {'entity': 'liranc6', 'mode': 'online', 'project': 'mrdiff', 'resume': 'None', 'run_name': 'None', 'id': 'None', 'save_code': False, 'resume_from': 'None'}, 'general': {'random_seed': 42, 'evaluate': False, 'tag': None, 'dataset': 'icentia11k', 'features': 'S', 'training_mode': 'ONE', 'interval': 1000}, 'optimization': {'learning_rate': 0.001, 'batc

In [76]:
exp = Exp_Main(args)

Use GPU: cuda:0


In [77]:
results_path = \
    "/home/liranc6/ecg_forecasting/liran_project/results/icentia11k/mrDiff/DDPM_icentia11k_ftS_sl105000_ll7500_pl7500_lr0.001_bs16_invFalse_itr0/25_10_2024_1822/best_checkp_mean_extra_r_beats.pth"

In [78]:
assert os.path.exists(results_path)

In [79]:
exp.args.resume_exp

Box({'resume': False, 'resume_from': 9, 'resume_optimizer': True, 'resume_epoch': 9, 'resume_metrics': True, 'resume_scheduler': True, 'resume_configuration': False, 'specific_chpt_path': '/home/liranc6/ecg_forecasting/liran_project/results/icentia11k/mrDiff/DDPM_icentia11k_ftS_sl105000_ll7500_pl7500_lr0.001_bs16_invFalse_itr0/21_10_2024_1424/logs/1_last_checkpoint.pth', 'was_resumed': False, 'model_start_training_time': 'None'})

In [80]:
# Reload the module
importlib.reload(liran_project.mrdiff.exp_main)

# Assuming `exp` is an existing instance of `Exp_Main`
exp.__class__ = liran_project.mrdiff.exp_main.Exp_Main

torch.cuda.empty_cache()
! gpustat

6104.37s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


chuck2                    Sun Oct 27 00:06:08 2024  535.183.01
[0] NVIDIA A100-SXM4-80GB | 33°C,   0 % | 48723 / 81920 MB | liranc6(48714M)


In [81]:
import torch
import sys

def measure_memory_usage(instance):
    memory_usage = {}
    for attr_name in dir(instance):
        if not attr_name.startswith('__') and not callable(getattr(instance, attr_name)):
            attr = getattr(instance, attr_name)
            # Measure memory usage of the attribute in bytes
            size_in_bytes = sys.getsizeof(attr)
            print(f"Attribute: {attr_name}, Size in bytes: {size_in_bytes}")
            memory_usage[attr_name] = size_in_bytes / (1024 ** 2)
            if isinstance(attr, torch.Tensor):
                # Measure additional GPU memory usage for tensors
                memory_before = torch.cuda.memory_reserved()
                attr_gpu = attr.to('cuda')
                memory_after = torch.cuda.memory_reserved()
                gpu_memory_usage = memory_after - memory_before
                print(f"Attribute: {attr_name}, GPU Memory in bytes: {gpu_memory_usage}")
                memory_usage[attr_name] += gpu_memory_usage / (1024 ** 2)
                attr_gpu.to('cpu')
    return memory_usage

# Measure memory usage of each attribute
memory_usage = measure_memory_usage(exp)
print(memory_usage)

Attribute: args, Size in bytes: 48
Attribute: dataloaders, Size in bytes: 64
Attribute: datasets, Size in bytes: 64
Attribute: device, Size in bytes: 24
Attribute: model_start_training_time, Size in bytes: 16
Attribute: set_models_using_meta, Size in bytes: 72
{'args': 4.57763671875e-05, 'dataloaders': 6.103515625e-05, 'datasets': 6.103515625e-05, 'device': 2.288818359375e-05, 'model_start_training_time': 1.52587890625e-05, 'set_models_using_meta': 6.866455078125e-05}


In [82]:
resume_configs = exp.args.resume_exp
resume_configs['resume'] = True
resume_configs['resume_configuration'] = True
resume_configs['resume_optimizer'] = False
resume_configs['resume_scheduler'] = False
resume_configs['resume_metrics'] = False
resume_configs['specific_chpt_path'] = results_path

exp.args.update_config_from_dict(resume_configs)


In [83]:
# iteration = exp.load_checkpoint(results_path)
# iteration

In [84]:
exp.args.testing.patients.end_patient = 5

In [85]:
# Convert Box object to dictionary
config_dict = exp.args.configs

# Access the configuration values using dictionary syntax
random_seed = config_dict['general']['random_seed']
tag = config_dict['general']['tag']
dataset = config_dict['general']['dataset']
features = config_dict['general']['features']

learning_rate = config_dict['optimization']['learning_rate']
batch_size = config_dict['optimization']['batch_size']

context_len = config_dict['training']['sequence']['context_len']
label_len = config_dict['training']['sequence']['label_len']
model = config_dict['training']['model_info']['model']
pred_len = config_dict['training']['sequence']['pred_len']
iterations = config_dict['training']['iterations']['itr']

inverse = config_dict['data']['inverse']

setting = f"{model}_{dataset}_ft{features}_sl{context_len}_ll{label_len}_pl{pred_len}_lr{learning_rate}_bs{batch_size}_inv{inverse}_itr{iterations}"
setting

'DDPM_icentia11k_ftS_sl0_ll105000_pl7500_lr0.001_bs16_invTrue_itr1'

In [86]:
fix_seed = random_seed
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

In [87]:
exp.datasets, exp.dataloaders = {}, {}
exp.args.testing.patients.end_patient = 1

In [88]:
# Reload the module
importlib.reload(liran_project.mrdiff.exp_main)
import mrDiff.utils.tools
importlib.reload(mrDiff.utils.tools)

# Assuming `exp` is an existing instance of `Exp_Main`
exp.__class__ = liran_project.mrdiff.exp_main.Exp_Main

torch.cuda.empty_cache()
! gpustat

exp.test(setting=setting,
         time_path='09_10_2024_1612',
         test = True,
         visualize = True,
         chpt_path=results_path
         )

6110.83s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


chuck2                    Sun Oct 27 00:06:15 2024  535.183.01
[0] NVIDIA A100-SXM4-80GB | 33°C,   0 % | 48723 / 81920 MB | liranc6(48714M)


creating_stats: 100%|██████████| 2/2 [00:05<00:00,  2.55s/it, time_elapsed=0:00:05]


loading model
Successfully loaded model from specific_chpt_path


test_pbar:   0%|          | 0/1 [00:05<?, ?it/s]


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.